In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
np.random.seed(1000)
import pathlib

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
print(data_dir)
data_dir = pathlib.Path(data_dir).with_suffix('')
print(data_dir)

228813984/228813984 [==============================] - 1s 0us/step
/root/.keras/datasets/flower_photos.tar
/root/.keras/datasets/flower_photos


In [4]:
batch_size=12
img_height=224
img_width=224

In [5]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    data_dir,validation_split=0.2,
    subset="training",
    seed=123, #the same set of images always go into the training, test, validation set.
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [6]:
val_ds=tf.keras.utils.image_dataset_from_directory(
    data_dir,validation_split=0.2,
    subset="validation",
    seed=123, #the same set of images always go into the training, test, validation set.
    image_size=(img_height,img_width),
    batch_size=batch_size
)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [7]:
#cheching the batch_size of training dataset
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(12, 224, 224, 3)
(12,)


In [8]:
normalization_layer = layers.Rescaling(1./255) #255 is constant pixel value

In [11]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [12]:
num_classes=print(len(class_names))

5


In [13]:
#model create (
#num_classes = len(class_names)
#model = Sequential([
#    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)), #input layer includes normalisation, ,sizes, color indication
#   layers.Conv2D(16, 3, padding='same', activation='relu'), #convolution layer    #16 - no. of filters,  3 - 3X3 filter size
#   layers.MaxPooling2D(), #pooling layer
#    layers.Conv2D(32, 3, padding='same', activation='relu'), #convolution layer    #32 - no. of filters,  3 - 3X3 filter size
#    layers.MaxPooling2D(), #pooling layer   #it is "2D" based on the dimention of the input image    #"1D" Pooling-text,  #"3D" pooling-3D images
#    layers.Conv2D(64, 3, padding='same', activation='relu'), #convolution layer    #64 - no. of filters,  3 - 3X3 filter size
#   layers.MaxPooling2D(), #pooling layer
#   layers.Flatten(), #converting the data into single-vector
#   layers.Dense(128, activation='relu'), #hidden layer1 with 128 neurons
#   layers.Dense(256, activation='relu'), #hidden layer2 with 256 neurons
#   layers.Dense(num_classes) #output layer  #num_classes gives me how many output layer i want.  i.,e = 5 .
#])

In [15]:
#model compile
#model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam', metrics=['accuracy'])

In [16]:
#inceptionv3 model - pretrained
InceptionV3_model=tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))

87910968/87910968 [==============================] - 0s 0us/step


In [17]:
from keras.models import Model

In [18]:
for layer in InceptionV3_model.layers[:-15]:
  layer.trainable = False
x=InceptionV3_model.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(units=512,activation='softmax')(x)
x=Dropout(0.3)(x)
x=Dense(units=512,activation='softmax')(x)
x=Dropout(0.3)(x)
output=Dense(units=5,activation='softmax')(x)
model=Model(InceptionV3_model.input, output)

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [20]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs=5
history=model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/5
245/245 [==============================] - 539s 2s/step - loss: 1.6045 - accuracy: 0.2459 - val_loss: 1.6020 - val_accuracy: 0.2398
Epoch 2/5
245/245 [==============================] - 514s 2s/step - loss: 1.6007 - accuracy: 0.2459 - val_loss: 1.6007 - val_accuracy: 0.2398
Epoch 3/5
134/245 [===============>..............] - ETA: 2:48 - loss: 1.5954 - accuracy: 0.2525

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range=range(epochs)
plt.figure(figsize=(8,8))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#vgg19

In [ ]:
pretrained_model1=tf.keras.applications.VGG19(input_shape = (224,224,3),
                        include_top = False,
                        weights ='imagenet')

In [ ]:
for layer in pretrained_model1.layers[:-15]:
  layer.trainable = False
x=pretrained_model1.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(units=512,activation='softmax')(x)
x=Dropout(0.3)(x)
x=Dense(units=512,activation='softmax')(x)
x=Dropout(0.3)(x)
output=Dense(units=5,activation='softmax')(x)
model=Model(pretrained_model1.input, output)

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=5
history1=history=model.fit(train_ds,validation_data=val_ds,epochs=epochs)

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs_range=range(epochs)
plt.figure(figsize=(8,8))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()